# Analysis 2: Foundations of modeling 2

## Higher-order functions: decorators

A **decorator** in Python allows a user to add new functionality to an
existing object (either a function or a class) without modifying its structure. With decorators,
functions are taken as the argument into another function and then called inside the wrapper
function.

This document contains:
- [Functions inside functions](#inside)
- [Functions returning functions](#returning)
- [Functions as arguments](#asarg)
- [Practice before decorators](#practice)
- [Simple decorator](#simpledec)
- [Syntactic sugar](#sugar)
- [Decorating built-in functions (optional)](#builtin)
- [Generic wrapper (optional)](#generic)
---
<a id='inside'></a>
### Functions inside functions
Python allows you to define a new function inside of another function. This concept doesn’t exist
in all programming languages (e.g. C++), and to understand how decorators work, we will
illustrate it with a few examples.
##### Example – inner function:

In [ ]:
def outer_function():
    print("Hello! This is the first line of the outer function.")
    
    def inner_function():
        print("This is inner function! Answering the call!")
    
    print("This is the last line of the outer function. Goodbye!")

outer_function()
#inner_function()

The example defines the non-parametric `outer_function` which prints text when called. Inside of
`outer_function` we define another non-parametric function `inner_function`. When running the
program only the “hello” and “goodbye” texts will be printed, but nothing related to `inner_function`.
That is because it is defined but not called.  Also, `inner_function` belongs to the namespace
of `outer_function` only. Therefore, calling `inner_function` from the main program will raise an
error. (You can uncomment the last line and test it yourself.)

Next, we modify `outer_function` and add a call to `inner_function`:

In [ ]:
def outer_function():
    print("Hello! This is the first line of the outer function.")
    
    def inner_function():
        print("This is inner function! Answering the call!")
    
    print("This is outer function calling inner function:")
    inner_function()
    print("This is the last line of the outer function. Goodbye!")
outer_function()

This shows that we can define and call inside functions at any place in outer functions. In this
example, we chose to have inner_funciton called between two print statements for clearer
illustration.
#### Experiment
- Change the position of inner_function call. Adjust the position of inner_function
definition if needed.

---
<a id='returning'></a>
### Functions returning functions
Like we did in examples with lambdas, we can create a function inside of another function
and then return it. For the illustration, we modify the previous example to return
`inner_function`.
##### Example – returning function:

In [ ]:
def outer_function():
    print("Hello! This is the first line of the outer function.")
    
    def inner_function():
        print("This is inner function! Answering the call!")
    
    print("This is the outer function, returning the inner function.")
    return inner_function

f = outer_function()
f()

When `outer_function` is called, it will return `inner_function`, which we assign to variable f.
Then, we can call `inner_function` using that variable from the main program.

---
<a id='asarg'></a>
### Functions as arguments
Functions can also be passed to other functions as arguments.
##### Example – passing function(s) as arguments:

In [ ]:
def alice():
    print("Hello this is Alice.")

def bob():
    print("Hello this is Bob.")

def eve(f):
    print("This is Eve, calling function:",f.__name__)
    f()
    
eve(alice)
eve(bob)

Here, we have function `eve` that takes function `alice` and then function `bob` as an argument.
When called, function `eve` prints the name of the function it was given as an argument, and
then calls that function.
#### Experiment
- Define another non-parametric function and pass it to function eve.
- Define one parametric function and pass it to function eve. What happens?

---
<a id='practice'></a>
### Practice before decorators
Before decorator examples, let’s make a final practice with inner functions.  Assume that you
were given a task to write a function that will calculate the grade for the multiple-choice exam,
consisting of $t$ questions (so typically $t=40$), 
with each question having one correct and three incorrect options to
choose from.  If a student answers $p$ questions correctly, their grade can be calculated using
the following formula:
$$
g(p) = \max
\left(
1,\,\,
\frac{p-\frac{t}{4}}{t-\frac{t}{4}}\cdot 10
\right)
$$
The term $\frac{t}{4}$ is subtracted on both sides to correct for the possibility to just guess the answer.

This formula can be implemented with the following function:

In [ ]:
def grade(points, totalpoints):
    result = (points - totalpoints/4) / (totalpoints - totalpoints/4) * 10
    return max(1.0, round(result,1))

print(grade(40,40)) # should return 10.0
print(grade(20,40)) # should return 3.3
print(grade(26,40)) # should return 5.3
print(grade(27,40)) # should return 5.7
print(grade(0,40)) # should return 1.0
print(grade(5,40)) # should return 1.0

This function works as intended, but assuming that all students have taken the same exam (with
the same number of questions), makes the second argument redundant. An alternative is to
create a function once for each exam, stating the number of questions, and have it return a
function where only the number of correct answers is needed.

In [ ]:
def make_grade(totalpoints):
    def grade(points):
        result = (points - totalpoints/4) / (totalpoints - totalpoints/4) * 10
        return max(1.0, round(result,1))
    return grade

Analysis2 = make_grade(40)

print(Analysis2(40))
print(Analysis2(20))
print(Analysis2(26))
print(Analysis2(27))
print(Analysis2(0))
print(Analysis2(5))

#### Experiment
- Try to write a function that returns another function based on the previous example. You
can start with a function that returns a linear function for the given slope and y-intercept
values.

---
<a id='simpledec'></a>
### Simple decorator
To understand decorators, we will investigate a simple problem concerning division of two
numbers $a$ and $b$.
##### Example – division by zero:

In [ ]:
def div(a, b):
    return a/b

print(div(6,2))
#print(div(6,0)) # raises an error

If we implement the function `div` to just return the result of $a/b$, it will work in all cases, except
when $b = 0$, as division by zero is not allowed. Uncommenting the last line will raise an
error.
One approach is to modify the function, check if $b=0$ and skip division if true.

In [ ]:
def div_fixed(a, b):
    if b==0:
        print("Division by zero is not allowed!")
        return None
    else:
        return a/b
    
print(div_fixed(6,2))
print(div_fixed(6,0))

But ... what if altering the function is not possible? Assume you are using a function that
someone else wrote, and you either don’t have the source code, or you just want to make a
minor alteration, without going deep into the implementation. Using our knowledge of writing
inner and outer functions, we can write a function that wraps around another.
##### Example – our first decorator:

In [ ]:
def div_with_check(func):
    def wrapper(a, b):
        if b == 0:
            return None
        else:
            return func(a, b)
    
    return wrapper

Let’s analyze this structure in detail. Function `div_with_check` is accepting one argument
which is expected to be a function. When called, `div_with_check` will go to the return statement
(as everything else was the definition of inner function `wrapper`) and return function `wrapper`,
that takes two arguments, as the result.

If `wrapper` is bound to a variable and called later via that variable, including passing of two
arguments, it will do the following: first, it will check if the second argument is equal to zero,
returning `None` if true. Otherwise, it will call function `func`, passing to it arguments `a, b`.

To wrap around another function, we can use the following call:

In [ ]:
div = div_with_check(div)

Essentially, this means that we are wrapping around function `div`, and using the wrapper avoid
division by zero. The complete code can be found below:

In [ ]:
def div_with_check(func):
    def wrapper(a, b):
        if b == 0:
            return None
        else:
            return func(a, b)
    return wrapper

def div(a, b):
    return a/b

print(div(6,2))
#print(div(6,0)) # raises an error
div = div_with_check(div)
print(div(6,2))
print(div(6,0)) # no longer raises an error

By using the decorator div_with_check, we altered the functionality of the function div, while
the original implementation remained unchanged.

---
<a id='sugar'></a>
### Syntactic sugar
Syntactic sugar is a syntax within a programming language that is designed to make things
easier to read or to express. It makes the language “sweeter” for human use. If you are writing
your own functions, you can use syntactic sugar to decorate them. In Python, this is done by
writing just above the function definition the at sign ‘@’ followed by the name of a decorator. We
will use the same example from above to illustrate.

In [ ]:
def div_with_check(func):
    def wrapper(a, b):
        if b == 0:
            return None
        else:
            return func(a, b)
    return wrapper

@div_with_check
def div(a, b):
    return a/b

print(div(6,2))
print(div(6,0)) # decorated div doesn't raise an error

---
<a id='builtin'></a>
### Decorating built-in functions (optional)
Often, you will want to decorate functions written by a third party. Although this example has
no practical value, we will make an Easter-egg to wrap around the built-in function sum, such
that, if it gets exactly 3 input values, we trigger a secret message “Analysis 2 is the best course
I ever had!”

**Important:** you must be careful how you address the arguments of a function you are wrapping
around. There is a generic way to do this, but for now, we will write a decorator explicitly for
one function, where we know which arguments it is taking. This can be accomplished by
consulting `help()`.

#### Example – Easter egg decorator
We will follow the same template as with the first decorator. Create a decorating function that
takes another function as an argument. Inside, create a wrapper accepting exactly the same
number and type of arguments, as the function we want to wrap around. Then, we implement
the specific behavior and return the wrapper.

In [ ]:
def easter_egg(func):
    def wrapper(iterable, start=0):
        if len(iterable) == 3:
            return "Analysis 2 is the best course I ever had!"
        else:
            return func(iterable, start)
    return wrapper

print(sum([1,3]))
print(sum([1,3,5]))
print(sum([1,3,5,7]))

sum = easter_egg(sum)
print(sum([1,3]))
print(sum([1,3,5]))
print(sum([1,3,5,7]))

#### Experiment
- Pick one of the built-in functions and alter its functionality with a decorator.

---
<a id='generic'></a>
### Generic wrapper
Finally, let’s make an attempt of writing a wrapper around functions for which we don’t know
the number and the type of arguments. We still use the same “template” for wrapper, but this
time we pass non-keyworded variable-length argument and keyworded variable-length
argument.

#### Example – Testing functions
A practical example would be to write a wrapper that can measure how long it took the
computer to execute our function. We do this by recording the time when the function started,
the time when the function ended, and output the difference as the duration in seconds. Between
that, we have to call the decorated function and also return it’s return value. Finally, the wrapper
must be returned.

In [ ]:
import time

def test(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        value = func(*args, **kwargs)
        end = time.perf_counter()
        duration = end - start
        print("%s running time: %.4f" %(func.__name__, duration))
        return value
    return wrapper

Let’s write a few functions to test.

In [ ]:
@test
def hello():
    print("Hello world!")

@test
def nested_loops(m=10, n=10, p=10):
    for i in range(m):
        for j in range(n):
            for k in range(p):
                print("Line:", i, j, k)

@test
def fact(n):
    res = 1
    for i in range(1,n+1):
        res *= i
    return res

And also decorate built-in function sum:

In [ ]:
sum = test(sum)

To see if the decorator works as expected, call them all:

In [ ]:
print(sum((1,2,3)))
hello()
nested_loops()  # Notice how printing to the standard output impacts the execution time.
x = fact(1000)
print(x)